# 1. Setup and Imports


In [ ]:
# Install required packages
!pip install -q roboflow ultralytics

In [ ]:
from roboflow import Roboflow
from google.colab import drive
import os
import json

In [ ]:
# Check GPU availability
!nvidia-smi 

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Read config file
with open('/content/drive/MyDrive/path_to_your_config.json', 'r') as config_file:
    config = json.load(config_file)

# Set up environment variables
os.environ['ROBOFLOW_API_KEY'] = config['roboflow_api_key']

HOME = os.getcwd()
print(f"Working directory: {HOME}")

Sun Sep 22 20:31:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 2. Dataset Preparation


In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

rf = Roboflow(api_key=os.environ['ROBOFLOW_API_KEY'])
project = rf.workspace(config['workspace']).project(config['project'])
version = project.version(config['version'])
dataset = version.download("yolov8")

!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!sed -i '$d' {dataset.location}/data.yaml
!echo -e "test: ../test/images\ntrain: ../train/images\nval: ../valid/images" >> {dataset.location}/data.yaml

# 3. Model Training


In [ ]:
%cd {HOME}

!yolo task=detect \
mode=train \
model=yolov8x.pt \
data={dataset.location}/data.yaml \
batch=4 \
epochs=50 \
imgsz=1280 \
plots=True

# 4. Model Validation


In [ ]:
%cd {HOME}

!yolo task=detect \
mode=val \
model={HOME}/runs/detect/train/weights/best.pt \
data={dataset.location}/data.yaml

/content
Ultralytics YOLOv8.2.98 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs
val: Scanning /content/datasets/Football-Tracking-3/valid/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100% 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:13<00:00,  3.46s/it]
                   all         49       1174      0.939      0.873      0.909      0.683
                  ball         45         45      0.915      0.644      0.707      0.375
            goalkeeper         38         39      0.886      0.897      0.947      0.772
                player         49        973      0.971       0.99      0.994      0.853
               referee         49        117      0.983      0.962      0.988      0.733
Speed: 26.3ms preprocess, 205.0ms inference, 0.0ms loss, 24.0ms postprocess per image
Results saved to runs/dete

# 5. Model Deployment


In [ ]:
project.version(config['version']).deploy(model_type='yolov8', model_path=f"{HOME}/runs/detect/train")

Dependency ultralytics==8.0.196 is required but found version=8.2.98, to fix: `pip install ultralytics==8.0.196`
Would you like to continue with the wrong version of ultralytics? y/n: y
View the status of your deployment at: https://app.roboflow.com/football-tracking-an83l/football-tracking-1yu0o/3
Share your model with the world at: https://universe.roboflow.com/football-tracking-an83l/football-tracking-1yu0o/model/3
